# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
import os.path
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import re

# Scraping Data

In [2]:
browser =  webdriver.Chrome(PATH)
##### Web scraper for infinite scrolling page #####
browser.get("https://www.onmap.co.il/en/homes/buy")
time.sleep(5)

/Users/hannahlang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [3]:
# identify scrolling element first i.e. the sidebar
scrolling_element_xpath = '/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div[1]/div'
scrolling_element= browser.find_element("xpath", scrolling_element_xpath)    

last_height = browser.execute_script("return arguments[0].scrollHeight", scrolling_element)

SCROLL_PAUSE_TIME = 2.0 
t = 0 

## Loop the scrolling until cannot scroll anymore
while True:
    print(t, end = ', ')
    # Scroll down to bottom of whatever is currently loaded
    browser.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', scrolling_element)
    t = t+1

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Check if more scrolling required 
    new_height = browser.execute_script("return arguments[0].scrollHeight", scrolling_element)
    if new_height == last_height:
        break
    last_height = new_height

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 

In [4]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [5]:
def collect_page_data(soup):
    #Listing prices
    prices = []
    for price in soup.find_all("span", {"class": "cWr2cxa0k3zKePxbqpw3L"}):
        prices.append(price.text)
    #Listing addresses
    addresses = []
    drop_idxs = []
    drop_idxs_extra = []
    for idx, address in enumerate(soup.find_all("div", {"class": "_2d0UdIoLkw2Kq1JbpZf3Z2"})):
        if 'Status' in address.text:
            drop_idxs.append(idx)
            drop_idxs_extra.append(idx + 1)
            addresses.append(address.text)
        else:
            addresses.append(address.text)
    addresses = list(np.delete(np.array(addresses), np.array(drop_idxs + drop_idxs_extra)))
    #Listing property type
    prop_types = []
    for prop_type in soup.find_all("div", {"class": "_368bz4_3ywt2f0d7eux_Li"}):
        prop_types.append(prop_type.text)
    prop_types = list(np.delete(np.array(prop_types), np.array(drop_idxs)))
    #Property features
    num_rooms = []
    flr_num = []
    mtrs = []
    parking = []
    for apt_feat in soup.find_all("div", {"class": "yHLZr2apXqwIyhsOGyagJ"}):
        feat_lst = apt_feat.find_all("span")
        num_rooms.append(feat_lst[0].text)
        flr_num.append(feat_lst[1].text)
        mtrs.append(feat_lst[2].text)
        if len(feat_lst)>3:
            parking.append(feat_lst[3].text)
        else:
            parking.append('NA')
    df = pd.DataFrame({'price': prices, 'address': addresses, 'property_type':prop_types,
                       'num_rooms': num_rooms, 'floor_num': flr_num, 
                       'meters': mtrs, 'parking': parking})
    return df

In [6]:
housing_data = collect_page_data(soup)

In [7]:
housing_data

,price,address,property_type,num_rooms,floor_num,meters,parking
0,"₪ 2,790,000",Shlomo Kaplanski St. Rishon LeTsiyon,Apartment,5,4,125,2
1,"₪ 6,000,000",King George St. Tel Aviv-Yafo,Apartment,4,1,110,NA
2,"₪ 11,900,000","Rothschild Boulevard 17,Tel Aviv-Yafo",Apartment,5,6,165,4
3,"₪ 5,420,000","Yoseftal St. 11,Bat Yam",Rooftop apt.,7,4,140,1
4,"₪ 3,750,000",Metula St. Ramat Gan,Penthouse,5,7,250,3
...,...,...,...,...,...,...,...
3740,"₪ 1,780,000",Dror St. Rishon LeTsiyon,Apartment,4,5,95,1
3741,"₪ 4,900,000",Nitsa Boulevard Netanya,Mini penthouse,7,13,260,3
3742,"₪ 2,850,000",Bnei Binyamin St. Netanya,Mini penthouse,5,7,151,2
3743,"₪ 2,450,000",Bnei Binyamin St. Netanya,Apartment,4,2,123,2


# Cleaning the DF

In [8]:
housing_data.head()

,price,address,property_type,num_rooms,floor_num,meters,parking
0,"₪ 2,790,000",Shlomo Kaplanski St. Rishon LeTsiyon,Apartment,5,4,125,2
1,"₪ 6,000,000",King George St. Tel Aviv-Yafo,Apartment,4,1,110,NA
2,"₪ 11,900,000","Rothschild Boulevard 17,Tel Aviv-Yafo",Apartment,5,6,165,4
3,"₪ 5,420,000","Yoseftal St. 11,Bat Yam",Rooftop apt.,7,4,140,1
4,"₪ 3,750,000",Metula St. Ramat Gan,Penthouse,5,7,250,3


## Adjusting the price numbers

In [9]:
def remove_non_numbers(exp):
    exp = re.sub('[^0-9]','', exp)
    return exp

In [10]:
housing_data['price'] = housing_data.price.apply(remove_non_numbers)

## Extracting the Address and City

### Finding main city names

In [23]:
def extract_main_city(address):
    address = re.sub('\'', '', address)
    split_address = address.split()
    if 'St.' in address:
        address = re.split("St.", address)
        town = address[-1].strip().split(',')[-1]
    elif ',' in address:
        address = re.split(',', address)
        town = address[-1].strip()
    elif 'Road' in address:
        address = re.split("Road", address)
        town = address[-1].strip()
    else:
        town = None
    return town

In [24]:
main_city_list = housing_data['address'].apply(extract_main_city).unique()

In [25]:
main_city_list

array(['Rishon LeTsiyon', 'Tel Aviv-Yafo', 'Bat Yam', 'Ramat Gan',
       'Netanya', 'Ashkelon', 'Beer Sheva', 'Mazkeret Batya', 'Sderot',
       None, 'Holon', 'Jerusalem', 'Beer Yaakov', 'Tiberias', 'Rehovot',
       'Petah Tikva', 'Beersheba', 'Lod', 'Herzliya', 'Haifa',
       'Migdal HaEmek', 'Ramla', 'Ashdod', 'Hod Hasharon', 'Nesher',
       'Yokneam Illit', 'feinberg', 'Azor', 'Yavne', 'Gedera',
       'Tirat Carmel', 'Gan Yavne', 'באר שבע', 'Or Yehuda', 'Ness Ziona',
       'Tel Aviv', 'Ariel', 'Kiryat Yam', 'Nahariya', 'Caesarea',
       'Hadera', 'Beit Zait', 'Givat Zeev', 'Nehalim', 'Bnei Brak',
       'Pardes Hanna-Karkur', 'Zikhron Yaakov', 'Givat Shmuel',
       'Givatayim', 'Ganei Tikva', 'Neria', 'Kdumim', 'Karnei Shomron',
       'אריאל', 'Eilat', 'Lehavim', 'Kiryat Ekron', 'Tverya',
       'Kiryat Bialik', 'Rosh Haayin', 'Yehud-Monosson', 'Ramat Yishai',
       'Kfar Baruch', 'Alonei Abba', 'אפרת', 'Kfar Yona', 'Even Yehuda',
       'Afula', 'Nahariyya', 'Acre', 'Yer

### Extracting street and city

In [51]:
def extract_city(address):
    address = re.sub('\'', '', address)
    split_address = address.split()
    if 'St.' in address:
        address = re.split("St.", address)
        street = address[0] + 'St.'
        town = address[-1].strip().split(',')[-1]
    elif ',' in address:
        address = re.split(',', address)
        street = address[0]
        town = address[-1].strip()
    elif 'Road' in address:
        address = re.split("Road", address)
        street = address[0] + 'Road'
        town = address[-1].strip()
    elif split_address[-1].strip() in main_city_list:
        street = ''.join(split_address[:-1])
        town = split_address[-1].strip()
    elif ' '.join(split_address[-2:]).strip() in main_city_list:
        street = ' '.join(split_address[:-2])
        town = ' '.join(split_address[-2:]).strip()
    else:
        street = 'NA'
        town = 'NA'
    return street, town

In [52]:
street, town = zip(*housing_data['address'].apply(extract_city))

In [53]:
np.unique(np.array(town))

array(['Acre', 'Afula', 'Akko', 'Alonei Abba', 'Arad', 'Ariel', 'Ashdod',
       'Ashkelon', 'Azor', 'Bat Hefer', 'Bat Yam', 'Beer Sheva',
       'Beer Yaakov', 'Beerotayim', 'Beersheba', 'Beit Zait',
       'Bet Shemesh', 'Bnei Brak', 'Caesarea', 'Eilat', 'Elad', 'Eli',
       'Even Yehuda', 'Gan Yavne', 'Ganei Tikva', 'Gedera',
       'Givat Shmuel', 'Givat Zeev', 'Givatayim', 'Hadera', 'Haifa',
       'Hamra', 'Harish', 'Hatzor HaGlilit', 'Herzliya', 'Hod Hasharon',
       'Holon', 'Immanuel', 'Jerusalem', 'Kadima Zoran', 'Karmiel',
       'Karnei Shomron', 'Kdumim', 'Kfar Baruch', 'Kfar Hananya',
       'Kfar Vradim', 'Kfar Yona', 'Kiryat Ata', 'Kiryat Bialik',
       'Kiryat Ekron', 'Kiryat Gat', 'Kiryat Motzkin', 'Kiryat Yam',
       'Lehavim', 'Lod', 'Mazkeret Batya', 'Mehola', 'Meitar',
       'Migdal HaEmek', 'Mitzpe Ramon', 'Modiin Ilit',
       'Modiin-Maccabim-Reut', 'NA', 'Nahariya', 'Nahariyya', 'Nehalim',
       'Neria', 'Nesher', 'Ness Ziona', 'Netanya', 'Netivot', 'Nil

In [54]:
housing_data['street'] = street

In [55]:
housing_data['city'] = town

# Exporting CSV

In [56]:
housing_data.to_csv('housing_data_2.csv')

In [57]:
housing_data

,price,address,property_type,num_rooms,floor_num,meters,parking,street,city
0,2790000,Shlomo Kaplanski St. Rishon LeTsiyon,Apartment,5,4,125,2,Shlomo Kaplanski St.,Rishon LeTsiyon
1,6000000,King George St. Tel Aviv-Yafo,Apartment,4,1,110,NA,King George St.,Tel Aviv-Yafo
2,11900000,"Rothschild Boulevard 17,Tel Aviv-Yafo",Apartment,5,6,165,4,Rothschild Boulevard 17,Tel Aviv-Yafo
3,5420000,"Yoseftal St. 11,Bat Yam",Rooftop apt.,7,4,140,1,Yoseftal St.,Bat Yam
4,3750000,Metula St. Ramat Gan,Penthouse,5,7,250,3,Metula St.,Ramat Gan
...,...,...,...,...,...,...,...,...,...
3740,1780000,Dror St. Rishon LeTsiyon,Apartment,4,5,95,1,Dror St.,Rishon LeTsiyon
3741,4900000,Nitsa Boulevard Netanya,Mini penthouse,7,13,260,3,NitsaBoulevard,Netanya
3742,2850000,Bnei Binyamin St. Netanya,Mini penthouse,5,7,151,2,Bnei Binyamin St.,Netanya
3743,2450000,Bnei Binyamin St. Netanya,Apartment,4,2,123,2,Bnei Binyamin St.,Netanya


In [58]:
housing_data[housing_data['city'] == 'NA']

,price,address,property_type,num_rooms,floor_num,meters,parking,street,city
37,6700000,Derech HaZayit Kfar Shmuel,Apartment,6,3,350,3,NA,NA
2311,1520000,Through a beacon Ma'ale Efrayim,Cottage/house,4,1,77,NA,NA,NA
2776,1500000,Rifts Bekaot,Villa,5,1,122,NA,NA,NA
2989,1300000,578 Migdalim,Villa,3,1,85,NA,NA,NA
